## LLM Agent Refinement

### Single Agent Refinement

We start with a single agent whose goal it is to accomplish the task of writing a blog post about Udemy.com.

We will ask the agent to refine its own work. This will work, but it will have some limitations... If we start asking the writer to refine too many aspects of the post, we'll start noticing that it will have a hard time considering them all without forgetting any aspect. 

In [63]:
# from openai import OpenAI
import pandas as pd
import autogen

In [64]:
api_key = pd.read_csv("~/tmp/chat_gpt/postman_key_1.txt", sep=" ", header=None)[0][0]
print("Don't be a fool and send your api key to GitHub!")

Don't be a fool and send your api key to GitHub!


In [65]:
# creating a Conversable Agent to accomlpish a simple task
# 1st thing to do is to create a LLM config that specifies which LLM we want to use
# model from the list of models provided by OpenAI https://platform.openai.com/docs/models/continuous-model-upgrades
llm_config = {
    "model": "gpt-4o",
    # "model": "gpt-3.5-turbo",
    "api_key": api_key
    }
print("Don't be a fool and send your api key to GitHub!")

Don't be a fool and send your api key to GitHub!


In [66]:
# defining our task
# topics: udemy.com // generative AI // machine learning // blockchains
task = '''
       Write a concise but engaging blogpost about
       udemy.com. Make sure the blogpost is
       within 200 words.
       '''

In [67]:
# creating a writer agent
# this writer agent will also be asked to refine its own work
writer = autogen.AssistantAgent(
    name="Writer",
    system_message="You are a writer. You write engaging and concise " 
        "blogposts (with title) on given topics. You must polish your "
        "writing based on the feedback you receive and give a refined "
        "version. Only return your final work without additional comments.",
    llm_config=llm_config,
)

To __get the single writer agent to work__ on this task we use the simple `generate_reply()` function.

In [68]:
reply = writer.generate_reply(messages=[{"content": task, "role": "user"}])
print(reply)

**Unlocking Knowledge: The Udemy Revolution**

In a world increasingly driven by technology and innovation, education has broken free from traditional classroom boundaries. Udemy.com, a pioneer in online learning, is at the forefront of this educational revolution, offering a gateway to myriad opportunities and insights.

Founded in 2010, Udemy has transformed the landscape of learning by providing affordable, accessible, and diverse courses across countless disciplines. Whether you're eager to dive into coding, enhance your business acumen, or explore a creative passion like photography or graphic design, Udemy's expansive library—comprised of over 183,000 courses taught by expert instructors—ensures that there's something for everyone.

The platform's appeal lies in its flexibility; students can learn at their own pace, revisiting materials as needed. This self-directed approach suits today’s busy lifestyle, making it easier for individuals to balance career growth with personal deve

### Two-agents Reflection: Refinement

We have a first result but we feel like it could use some refinement. The writer agent itself will have a hard time covering all the different aspects we want to have refined therefore to address this limitation we use a sequential chat of agents.

Within this __sequential chat of agents__ each will be specialized in refining a different aspect of the blogpost.

In the following example we get two agents to work together to refine the result (the blog post) of the first agent.

__This time we're going to create a chat with two agents:__
* The Writer who's going to write the blogpost
* The Critic who's going to reflect on the work of the writer and provide constructive criticism

Since each agent will be focused on a single aspect of the task, this will give us a better result for each aspect, which should yield a task that is better fulfilled. 

The agents will acomplish the same task, so we won't need to update that, but we need to re-define our agents:

In [69]:
# writer agent, similar to the one we've already defined above
writer = autogen.AssistantAgent(
    name="Writer",
    system_message="You are a writer. You write engaging and concise " 
        "blogposts (with title) on given topics. You must polish your "
        "writing based on the feedback you receive from other agents and give a refined "
        "version. Only return your final work without additional comments.",
    llm_config=llm_config,
)

In [70]:
critic = autogen.AssistantAgent(
    name="Critic",
    llm_config=llm_config,
    system_message="You are a critic. You review the work of "
                "the writer and provide constructive "
                "feedback to help improve the quality of the content.",
)

We will initiate the chat by getting the critic to send a message, the task we specified earlier, to the writer. The writer will then answer the critic with a first proposal, the critic will provide constructive criticism, and they will do this for two rounds.

We'll specify that the summary method is the last message, because that should be the last version of refined article. This will allow us to easily pass this result to another app or function that could then publish it or do what we want to do with this blogpost.

In [71]:
chat_result = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2, # we predefine the number of turns till we get the final result
    summary_method="last_msg" # we don't want a summary of the whole chat, just the last (final) version
)

Critic (to Writer):


       Write a concise but engaging blogpost about
       udemy.com. Make sure the blogpost is
       within 200 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

**Unlocking Learning Opportunities with Udemy**

In today's fast-paced digital world, continuous learning is a must. Enter Udemy.com, an expansive online platform revolutionizing education by making it accessible and adaptable for everyone. Founded in 2010, Udemy has become a leader in online learning, offering an impressive array of over 155,000 courses across various domains such as technology, business, arts, and personal development. 

What sets Udemy apart is its diverse community of instructors. Professionals and industry experts from around the world create content-rich courses, providing learners with practical, real-world knowledge. Whether you're looking to enhance your skills for career advancement, explore a new hobby, or gai

In [72]:
# exploring the last message which is our final version of the blogpost through the summary
import pprint

In [73]:
pprint.pprint(chat_result.summary)

('**Discover New Horizons with Udemy**\n'
 '\n'
 'Have you ever wondered how to keep up with the rapidly evolving world around '
 'you? With online education booming, Udemy.com stands out as a game-changer '
 'by democratizing learning for everyone. Since its inception in 2010, Udemy '
 'has become a powerhouse in online education, boasting an astounding '
 'collection of over 155,000 courses in fields like technology, business, art, '
 'and more.\n'
 '\n'
 'What makes Udemy truly unique is its international community of instructors '
 'who are experts in their fields, offering hands-on, practical knowledge that '
 'you can apply immediately. Imagine learning programming from a software '
 'engineer or honing your business skills with an experienced '
 'entrepreneur—Udemy makes this a reality.\n'
 '\n'
 'Designed with flexibility in mind, Udemy allows you to learn at your own '
 "pace. Whether you're commuting, on a lunch break, or relaxing at home, you "
 'can dive into courses that f

### Nested chats: multi-agent refinement

__Additional reviewers:__

We can now improve our refinement by using multiple agent that each specialize in a specific type of criticism and refinement. We're going to add 3 different types of critics:
* A **SEO reviewer**: This reviewer will provide improvements to optimize the content produced for search engines
* A **Legal reviewer**: This reviewer will provide criticisms to ensure that the content produced is legally compliant
* An **Ethics reviewer**: This reviewer will ensure that the content is ethically sound

The right way to think about this is that all these agents are virtually one agent, the critic. What will happen is that they will all provide their criticism for various aspects of the blogpost and report them back to the critic. The critic will then share them with the writer to get a better version. Now this might be a lot of criticism information, so we'll have to find a way to make it succint and clear for the writer, we'll see how to do that next;  
but first let's first these 3 additional reviewer agents:

In [74]:
SEO_reviewer = autogen.AssistantAgent(
    name="SEO Reviewer",
    llm_config=llm_config,
    system_message="You are an SEO reviewer, known for "
        "your ability to optimize content for search engines, "
        "ensuring that it ranks well and attracts organic traffic. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)
SEO_reviewer

In [75]:
legal_reviewer = autogen.AssistantAgent(
    name="Legal Reviewer",
    llm_config=llm_config,
    system_message="You are a legal reviewer, known for "
        "your ability to ensure that content is legally compliant "
        "and free from any potential legal issues. "
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)
legal_reviewer

In [76]:
ethics_reviewer = autogen.AssistantAgent(
    name="Ethics Reviewer",
    llm_config=llm_config,
    system_message="You are an ethics reviewer, known for "
        "your ability to ensure that content is ethically sound "
        "and free from any potential ethical issues. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role. ",
)
ethics_reviewer

### Meta Reviewer Agent

Since all of these reviews is going to be a lot of data, we are going to add another agent who is going to play the role of a meta-reviewer, a reviewer that is going to gather the reviews of all other reviewers and provide a summary to the writer.

The meta-reviewer will be provided by the context (summary of each previous review in a structured format) and will use the summary of each previous review to propose a final set of reviews that gathers all previous ones to the critic, who will then report them to the writer.

In [77]:
meta_reviewer = autogen.AssistantAgent(
    name="Meta Reviewer",
    llm_config=llm_config,
    system_message="You are a meta reviewer, you aggregate and review "
    "the work of other reviewers and give a final suggestion on the content.",
)
meta_reviewer

### Chat Orchestration with Nested Chats: Multi Agent Refinement

The way our chat is going to work is that when the writer will answer the critic, this time, the critic will actually trigger a series of nested chats with each specialized reviewer (Critic -> Reviewer and then Reviewer -> Critic). 

We are also going to request from each reviewer that they send back their review in a specific format. Each review will send back a LLM generated summary of their review in the following JSON format:  
`{'Reviewer': '', 'Review': ''}`  
This will make it easier for the meta-reviewer to summarize all reviews.

We will also define here is a simple function called `reflection_message()` that will create the following message:
```
Review the following content.

"BLOGPOST PROPOSED BY WRITER"
```
We will call this function to create the message sent by the Critic to each specialized reviewer sequentially.

The reflection_message() function defined here will create the following message:
'''
Review the following content.
"BLOGPOST PROPOSED BY WRITER"
'''
This function will be called to create the message sent by the Critic to each specialized reviewer sequentially.    

In [78]:
#def reflection_message(recipient, messages, sender, config):
#    return f'''Review the following content. 
#            \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}'''
def reflection_message(recipient, messages, sender, config):
    return f"Review the following content. \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}"

We are now going to define a new type of chat, a nested chat, that will trigger when the critic receives an answer. You can think about it like the inner monologue the Critic is having with other Reviewers that will help him provide the best possible criticism of the blogpost written by the reviewer. This is the structure our chat will follow:

**Main chat**:  
1. Critic -> Writer : Initial task (*"Write a concise but engaging blogpost ..."*)
2. Writer -> Critic : First version of the `blogpost`, this will trigger the **nested chat**

**Nested chat**:
1. __Critic__ -> SEO reviewer: *"Review the following content: `blogpost`"*
2. SEO reviewer -> Critic: `SEO review` with context `{'Reviewer': '', 'Review': ''}` 
3. Critic -> Legal reviewer: *"Review the following content: `blogpost`"*
4. Legal reviewer -> Critic: `Legal review` with context `{'Reviewer': '', 'Review': ''}` 
5. Critic -> Ethics reviewer: *"Review the following content: `blogpost`"*
6. Ethics reviewer -> Critic: `Ethics review` with context `{'Reviewer': '', 'Review': ''}`
7. Critic (<span style="color:green">has as this point received all three reviews</span>) -> Meta reviewer: *"Aggregrate feedback from all reviewers and give final suggestions on the writing."*
8. Meta reviewer -> Critic: Summary of all reviews with all contexts `{'Reviewer': '', 'Review': ''}`

**Enf of nested chat**

**Back to the main chat**:
<br/><span style="color:green">What the critic got out of the nested chat is the final result of the meta review.</span>
1. Critic -> Writer : Summary of all reviews with all contexts `{'Reviewer': '', 'Review': ''}`
2. Writer -> Critic : Refined version of the blogpost based on all reviews.

Since we've already seen how to define chats one by one, we'll define our nested chat all at once in a list this time:

In [79]:
review_chats = [ # This is our nested chat
    {
     "recipient": SEO_reviewer, 
     "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": 
        {
        "summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}. Here Reviewer should be your role",
        },
     "max_turns": 1},
    
    {
     "recipient": legal_reviewer, 
     "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {
            "summary_prompt": "Return review into as JSON object only:"
            "{'Reviewer': '', 'Review': ''}",
        },
     "max_turns": 1},
    
    {"recipient": ethics_reviewer, 
     "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'reviewer': '', 'review': ''}",},
     "max_turns": 1},
    
     {"recipient": meta_reviewer, 
      "message": "Aggregrate feedback from all reviewers and give final suggestions on the writing.", 
      "max_turns": 1},
]
review_chats

[{'recipient': <autogen.agentchat.assistant_agent.AssistantAgent at 0x10fba5b20>,
  'message': <function __main__.reflection_message(recipient, messages, sender, config)>,
  'summary_method': 'reflection_with_llm',
  'summary_args': {'summary_prompt': "Return review into as JSON object only:{'Reviewer': '', 'Review': ''}. Here Reviewer should be your role"},
  'max_turns': 1},
 {'recipient': <autogen.agentchat.assistant_agent.AssistantAgent at 0x10fba6ae0>,
  'message': <function __main__.reflection_message(recipient, messages, sender, config)>,
  'summary_method': 'reflection_with_llm',
  'summary_args': {'summary_prompt': "Return review into as JSON object only:{'Reviewer': '', 'Review': ''}"},
  'max_turns': 1},
 {'recipient': <autogen.agentchat.assistant_agent.AssistantAgent at 0x10fba4560>,
  'message': <function __main__.reflection_message(recipient, messages, sender, config)>,
  'summary_method': 'reflection_with_llm',
  'summary_args': {'summary_prompt': "Return review into as 

* Note how the `message` for each nested chat is going to be constructed by the `reflection_message() function we previously defined
* Note how each specialized reviewer will send back their review in the requested JSON format `{'reviewer': '', 'review': ''}`

We now need to save and register this nested chat as a chat that will be **<span style="color:green">only</span> triggered when the writer will contact the critic**:

In [80]:
# registering the nested chat
# review_chats, # list of sub-chats
# trigger=writer, # this chat will be ONLY triggered when the writer contacts the critic
critic.register_nested_chats(
    review_chats,
    trigger=writer,
)

### Defining the Main Chat

Ok, we are now ready to start this chat. We will start this with the main chat that will trigger the Critic's nested chat as soon as the writed send back an first proposal blogpost answer to the critic.

Pay attention to the order in which the exchanges will happen and feel free to go back to the orchestration structure presented above to ensure that you understand how the nested chat works:

In [81]:
chat_results = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


       Write a concise but engaging blogpost about
       udemy.com. Make sure the blogpost is
       within 200 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

**Unlocking Learning Opportunities with Udemy**

In today's fast-paced digital world, continuous learning is a must. Enter Udemy.com, an expansive online platform revolutionizing education by making it accessible and adaptable for everyone. Founded in 2010, Udemy has become a leader in online learning, offering an impressive array of over 155,000 courses across various domains such as technology, business, arts, and personal development. 

What sets Udemy apart is its diverse community of instructors. Professionals and industry experts from around the world create content-rich courses, providing learners with practical, real-world knowledge. Whether you're looking to enhance your skills for career advancement, explore a new hobby, or gai

/Users/juergenkober/Projects/Python+Rust/_github-repos/working-with-financial-data/py-building-llm-agent-teams/env/lib/python3.12/site-packages/autogen/agentchat/conversable_agent.py:1264: UserWarning: Cannot extract summary using reflection_with_llm: Error code: 400 - {'error': {'message': "Invalid 'messages[1].name': string does not match pattern. Expected a string that matches the pattern '^[a-zA-Z0-9_-]+$'.", 'type': 'invalid_request_error', 'param': 'messages[1].name', 'code': 'invalid_value'}}. Using an empty str as summary.
  warnings.warn(


Legal Reviewer (to Critic):

I am a legal reviewer. Here are my recommendations to ensure the content is legally compliant:

1. **Accuracy of Claims**: Verify that all claims about the number of courses and the flexibility of learning on Udemy are accurate and up-to-date to avoid misleading representations, and confirm that the named features, like lifetime access and affordability, align with current terms and conditions.

2. **Intellectual Property**: Ensure that no copyrighted materials, trademarks, or other intellectual property are used without proper attribution or permission, especially in logos or phrases like “Unlocking Learning Opportunities with Udemy.”

3. **Terms and Conditions**: Clearly state or hyperlink to Udemy’s terms of use or policies related to access and course purchase limitations, especially on aspects like promotions or refunds, to preempt any user disputes.

--------------------------------------------------------------------------------

********************

## Why does this matter?

One of the main weaknesses of LLMs like chatGPT, as you probably have noticed yourself if you've been using them, is that:
* when provided with complex tasks composed of several requirements, the LLM will most likely not accomplish all requirements properly: with nested chats, you can ensure that all requirements are all considered.
* LLMs very rarely directly provide the result you want in one step, you then have to keep asking them for specific refinements on the initial content: with nested chats, if you often ask for the same type of refinements, you can automatize this process to always accomplish these refinements.